# Parse Announcements

In [ ]:
from web3 import Web3
import pandas as pd

In [ ]:
with open("infurakey.txt", "r") as file:
    APIKEY = file.read().strip()

In [ ]:
infura = f"https://mainnet.infura.io/v3/{APIKEY}"
infura = "http://127.0.0.1:8545"

In [ ]:
w3 = Web3(Web3.HTTPProvider(infura))
assert w3.isConnected()

In [ ]:
event_signature_transfer = w3.keccak(text="Announcement(address,uint256,uint256)").hex()

blocknow = w3.eth.get_block_number()
    
    
event_filter = w3.eth.filter({
    "fromBlock": blocknow - 100,
    "toBlock": blocknow, 
    'topics': [event_signature_transfer]})
#transfer_events = w3.eth.get_filter_changes(event_filter.filter_id)
logs = w3.eth.get_filter_logs(event_filter.filter_id)

In [ ]:
try:
    knownHashes = set(pd.read_csv("knownhashes.txt")["txhash"].tolist())
    print(knownHashes)
    print("loaded")
except:
    knownHashes = set()
    print("not loaded")
    

try:
    sharedSecrets = pd.read_csv("sharedSecrets.txt")
    print("loaded")
except:
    sharedSecrets = pd.DataFrame(columns=["txhash", "stealthaddr", "pubkey_X", "pubkey_Y"])
    print("not loaded")

for log in logs:
    txhash = log["transactionHash"].hex()[0:30]
    if txhash in knownHashes:
        continue
    xy = []
    for ix, topic in enumerate(log["topics"]):
        if ix == 0:
            continue
        if ix == 1:
            stealthAddress = "0x" + topic.hex()[-40:]
        else:
            xy.append(int(topic.hex(),16))
            
    sharedSecrets.loc[len(sharedSecrets),("txhash", "stealthaddr", "pubkey_X", "pubkey_Y")] = txhash, stealthAddress, xy[0], xy[1]
    knownHashes.add(txhash)
    
sharedSecrets.to_csv("sharedSecrets.txt", index=None)
pd.DataFrame(knownHashes, columns=["txhash"]).to_csv("knownhashes.txt", index=None)